In [11]:
import numpy as np
from itertools import product
from scipy.stats import t, f

### Шаг 1: Масштабирование данных

In [12]:
def scale_factor(val, lower, upper):
    center = (upper + lower) / 2
    range_val = (upper - lower) / 2
    return (val - center) / range_val

### Генерация матрицы плана

In [13]:
def generate_planning_matrix():
    levels = [-1, 1]
    factors = list(product(levels, repeat=4))
    
    matrix = []
    for comb in factors:
        row = list(comb)
        # Взаимодействия до 4-й степени
        row += [
            comb[0] * comb[1], comb[0] * comb[2], comb[0] * comb[3],
            comb[1] * comb[2], comb[1] * comb[3], comb[2] * comb[3],
            comb[0] * comb[1] * comb[2], comb[0] * comb[1] * comb[3],
            comb[0] * comb[2] * comb[3], comb[1] * comb[2] * comb[3],
            comb[0] * comb[1] * comb[2] * comb[3]
        ]
        matrix.append(row)
    
    return np.array(matrix)

### Шаг 2: Подготовка данных y

In [14]:
y = np.array([
    (-9.79 - 11.10 - 10.12) / 3,   # Эксперимент 1
    (1.51 + 1.53 + 1.26) / 3,      # Эксперимент 2
    (-23.81 - 24.97 - 23.58) / 3,  # Эксперимент 3
    (25.53 + 22.75 + 27.30) / 3,   # Эксперимент 4
    (-8.02 - 6.70 - 7.80) / 3,     # Эксперимент 5
    (5.32 + 6.12 + 6.06) / 3,      # Эксперимент 6
    (-10.41 - 11.05 - 9.47) / 3,   # Эксперимент 7
    (-14.24 - 13.02 - 14.51) / 3,  # Эксперимент 8
    (-22.01 - 19.10 - 20.14) / 3,  # Эксперимент 9
    (-16.74 - 17.30 - 18.41) / 3,  # Эксперимент 10
    (12.40 + 11.80 + 12.76) / 3,   # Эксперимент 11
    (0.67 + 0.70 + 0.75) / 3,      # Эксперимент 12
    (-2.10 - 2.26 - 2.44) / 3,     # Эксперимент 13
    (-4.90 - 4.71 - 4.28) / 3,     # Эксперимент 14
    (-1.37 - 1.43 - 1.64) / 3,     # Эксперимент 15
    (76.04 + 85.23 + 91.92) / 3    # Эксперимент 16
])

### Шаг 3: Построение уравнения регрессии

In [15]:

def calculate_regression_coefficients(X, y):
    # Добавляем единичный столбец для свободного члена
    X = np.hstack((np.ones((X.shape[0], 1)), X))
    # Решаем систему: X.T * X * b = X.T * y
    b = np.linalg.lstsq(X, y, rcond=None)[0]
    return b

### Шаг 4: Проверка значимости коэффициентов (Стьюдент)

In [16]:
def check_significance(b, y, X):
    X = np.hstack((np.ones((X.shape[0], 1)), X))
    y_pred = X @ b
    residuals = y - y_pred
    variance = np.var(residuals, ddof=1)
    standard_error = np.sqrt(variance / len(y))
    
    t_critical = t.ppf(0.975, df=len(y) - len(b))
    significant = [i for i, coeff in enumerate(b) if np.abs(coeff / standard_error) > t_critical]
    
    return significant, t_critical

### Шаг 5: Пересчёт модели с учётом значимых коэффициентов

In [17]:
def update_model(b, X, y, significant_indices):
    X_new = X[:, significant_indices]
    X_new = np.hstack((np.ones((X_new.shape[0], 1)), X_new))
    b_new = np.linalg.lstsq(X_new, y, rcond=None)[0]
    return b_new, X_new

### Шаг 6: Проверка адекватности модели (Фишер)

In [24]:
# Проверка адекватности (Фишер)
def check_adequacy(b, X, y):
    y_pred = X @ b
    residuals = y - y_pred
    S2_residual = np.var(residuals, ddof=1)
    S2_y = np.var(y, ddof=1)
    F_calculated = S2_y / S2_residual
    
    # Степени свободы для числителя и знаменателя
    dfn = len(y) - len(b)  # степень свободы остаточной дисперсии
    dfd = len(b) - 1       # степень свободы модели
    
    F_critical = f.ppf(0.95, dfn, dfd)
    
    return F_calculated, F_critical

### Запуск всего процесса

In [25]:
planning_matrix = generate_planning_matrix()
X = planning_matrix

### Рассчитываем коэффициенты

In [26]:
b = calculate_regression_coefficients(X, y)
print("Коэффициенты регрессии:")
print(b)

Коэффициенты регрессии:
[1.088125   5.30520833 5.17604167 8.00979167 9.10270833 7.435625
 9.58270833 0.25145833 0.396875   2.55229167 5.89270833 4.46395833
 8.971875   3.31895833 3.018125   9.83020833]


### Проверяем значимость

In [29]:
significant, t_critical = check_significance(b, y, X)
print("Значимые коэффициенты (индексы):", significant)

Значимые коэффициенты (индексы): []


### Пересчитываем модель с учётом значимости

In [30]:
b_new, X_new = update_model(b, X, y, significant)
print("Обновлённые коэффициенты регрессии (после исключения):", b_new)

Обновлённые коэффициенты регрессии (после исключения): [1.088125]


### Проверка адекватности

In [31]:
F_calculated, F_critical = check_adequacy(b_new, X_new, y)
print("F-значение:", F_calculated)
print("F-critical:", F_critical)
print("Модель адекватна:", F_calculated < F_critical)

F-значение: 1.0000000000000002
F-critical: nan
Модель адекватна: False
